In [1]:

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
import skimage

In [21]:
# data loading
data = []
path = "../data/datasets/train_set/"
classes = os.listdir(path)


def decode_img(img):
    # reading image from filepath
    I = skimage.io.imread(img)

    # check if it is in vertical orientatioon
    if I.shape[0] < I.shape[1]:
        I = I.T
    # add channel dim
    I = np.expand_dims(I, axis=-1)
    I = np.repeat(I, 3, axis=-1)
    # resize and return
    return tf.image.resize(I, [224, 224])


for class_name in tqdm(classes):
    files = os.listdir(path + class_name)
    class_images = np.zeros((len(files), 224, 224,3),dtype=np.int32)
    for i, file in enumerate(files):
        I = decode_img(path + class_name + "/" + file)
        class_images[i] = I
    data.append(class_images)

print("Done")

100%|██████████| 21/21 [04:26<00:00, 12.71s/it]

21
(580, 224, 224, 3)
(556, 224, 224, 3)
(563, 224, 224, 3)
(566, 224, 224, 3)
(566, 224, 224, 3)
(618, 224, 224, 3)
(555, 224, 224, 3)
(551, 224, 224, 3)
(558, 224, 224, 3)
(503, 224, 224, 3)
(400, 224, 224, 3)
(295, 224, 224, 3)
(592, 224, 224, 3)
(551, 224, 224, 3)
(565, 224, 224, 3)
(577, 224, 224, 3)
(584, 224, 224, 3)
(529, 224, 224, 3)
(390, 224, 224, 3)
(373, 224, 224, 3)
(377, 224, 224, 3)
Done


In [50]:
print(len(data))
total_lengths = np.zeros(len(data),dtype=np.int32)
print(total_lengths.shape)
for i,x in enumerate(data):
    #print(x.shape[0])
    total_lengths[i] = x.shape[0]

#print(total_lengths)


21
(21,)
580
556
563
566
566
618
555
551
558
503
400
295
592
551
565
577
584
529
390
373
377
[580 556 563 566 566 618 555 551 558 503 400 295 592 551 565 577 584 529
 390 373 377]


In [38]:
# model
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras import layers, Model, Input
width_shape = 224
height_shape = 224

image_input = Input(shape=(width_shape, height_shape, 3))
model = EfficientNetV2L(input_tensor=image_input, include_top=False,weights='imagenet')




In [65]:
start = 0
for i in range(len(data[0:2])):
    preds = model.predict(data[i])
    preds = preds.reshape(total_lengths[i],preds.shape[1]*preds.shape[2]*preds.shape[3])
    stop = total_lengths[i] + start
    arr_shape = preds.shape
    arr = np.memmap("database.mmap",
                    dtype='float32',
                    mode='w+',
                    shape=arr_shape)

    arr[start:stop] = preds
    start = stop
    arr.flush(); del arr
    

    

19/19 [==============================] - 15s 817ms/step


OSError: [Errno 22] Invalid argument: 'database.mmap'

In [ ]:
def load_memmap_data(path, shape,
                     append_extra_length=None,
                     shape_only=False,
                     display=True):
    """
    Load data and datashape from the file path.

    • Get shape from [source_dir/fname_shape.npy}.
    • Load memmap data from [source_dir/fname.mm].

    Parameters
    ----------
    source_dir : (str)
    fname : (str)
        File name except extension.
    append_empty_length : None or (int)
        Length to appened empty vector when loading memmap. If activate, the
        file will be opened as 'r+' mode.
    shape_only : (bool), optional
        Return only shape. The default is False.
    display : (bool), optional
        The default is True.

    Returns
    -------
    (data, data_shape)

    """
    path_data = path
    
    data = np.memmap(path_data, dtype='float32', mode='r',
                         shape=(shape[0], shape[1]))
    if display:
        print(f'Load {shape[0]:,} items from \033[32m{path_data}\033[0m.')
    return data

In [ ]:
emb = load_memmap_data("database.mmap",(total_lengths,62720))

TypeError: only size-1 arrays can be converted to Python scalars